This notebook compares the results of genome-wide boundary CTCF perturbations with DNA sequence.

First, results are loaded from h5 and filtered based on whether a given boundary was responsive to CTCF motif mutagenesis.

Sites are further filtered by jaspar CTCF score.

Sequences around each site are loaded and visualized.

The TGCAxTTC motif is visible by eye in strong sequences, but only appears in ~1/4 of sequences.

In [1]:
import matplotlib.pyplot as plt
import bioframe
import pandas as pd
import numpy as np
import h5py
import os
from scipy.stats import spearmanr, pearsonr
import scipy.cluster
import pysam

from matplotlib import colors

cmap_purple_green_GC = colors.ListedColormap(
    ["mistyrose", "darkseagreen", "plum", "beige"]  # a  # c  # g  # t
)
cmap_rhee = colors.ListedColormap(
    ["red", "blue", "yellow", "green"]  # a red  # c blue  # g yllow  # t green
)

cmaps = [cmap_purple_green_GC, cmap_rhee]

In [2]:
import akita_utils
from akita_utils.utils import (
    h5_to_df,
    filter_by_chrmlen,
    dna_1hot,
    dna_1hot_index,
    dna_1hot_GC,
    dna_rc,
)

ImportError: cannot import name 'h5_to_df' from 'akita_utils.utils' (/home1/kamulege/akita_utils/akita_utils/utils.py)

### Load & filter genomic deletion SCD for boundary CTCFs

In [ ]:
df = akita_utils.filter_boundary_ctcfs_from_h5(
    h5_dirs="/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model*/scd.h5",
    score_key="SCD",
    threshold_all_ctcf=5,
)

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.scatter(df["score_10k"], df["score_all_ctcf"], 5, alpha=0.2)
plt.loglog()
plt.xlabel("score mut 10k")
plt.ylabel("score all CTCF")

plt.subplot(122)
plt.scatter(df["score_10k"], df["SCD"], 5, alpha=0.2)
plt.loglog()
plt.xlabel("score mut 10k")
plt.ylabel("SCD individual CTCF")
plt.tight_layout()

In [ ]:
# filter by rmsk
sites = akita_utils.filter_by_rmsk(df)

## add scores for jaspar core motif to table of sites
ctcf_motifs = bioframe.read_table(
    "/project/fudenber_735/motifs/mm10/jaspar/MA0139.1.tsv.gz",
    schema="jaspar",
    skiprows=1,
)
sites = sites.merge(
    ctcf_motifs,
    how="left",
    left_on=["chrom", "start_2", "end_2"],
    right_on=["chrom", "start", "end"],
)

sites_unfiltered = sites.copy()

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.scatter(sites["score"], sites["SCD"], 10, alpha=0.5)
plt.semilogy()
plt.xlabel("core motif jaspar score")
plt.ylabel("genomic deletion SCD")

plt.subplot(122)
plt.scatter(sites["score"], sites["INS-64"], 10, alpha=0.5)
plt.xlabel("core motif jaspar score")
plt.ylabel("ins-64 deletion score")

plt.tight_layout()

In [ ]:
## get sites with similar core motif strengths
# sites = sites.query('(score > 800) and (score < 850)')

sites = sites_unfiltered.query("(score > 825) and (score < 875)").copy()
sites.reset_index(drop=True, inplace=True)

In [ ]:
## split sites by strong vs weak, here using insulation score (ins-64)
sites_weak = sites["INS-64"].values > -0.01
sites_strong = sites["INS-64"].values < -0.03

sites_weak = np.where(sites_weak)[0]
sites_strong = np.where(sites_strong)[0]
print("num weak", len(sites_weak), ", num strong", len(sites_strong))

num_sites = 200
sites_weak = sites_weak[
    np.sort(np.random.choice(len(sites_weak), num_sites, replace=False))
]
sites_strong = sites_strong[
    np.sort(np.random.choice(len(sites_strong), num_sites, replace=False))
]
sites_strong = sites.loc[sites_strong].reset_index(drop=True)
sites_weak = sites.loc[sites_weak].reset_index(drop=True)

In [ ]:
def get_seqs(
    sites,
    genome_file="/project/fudenber_735/genomes/mm10/mm10.fa",
    extent_bp=100,
    verbose=False,
):
    """
    Get and orient DNA sequence around motifs, and return arrays with various transformation:
    into 1hot, index, and binary (GC).
    """
    genome_open = pysam.Fastafile(genome_file)

    seqs = []
    seqs_1hot = []
    seqs_index = []
    seqs_GC = []
    for i in range(len(sites)):
        if verbose:
            if np.mod(i, 50) == 0:
                print(i)
        chrm, start, end, strand = sites.loc[i][
            ["chrom", "start_2", "end_2", "strand_2"]
        ]
        start = start - extent_bp
        end = end + extent_bp
        seq = genome_open.fetch(chrm, start, end).upper()
        if strand == "-":
            seq = dna_rc(seq)
        seqs.append(seq)
        seqs_1hot.append(dna_1hot(seq))
        seqs_index.append(dna_1hot_index(seq))
        seqs_GC.append(dna_1hot_GC(seq))
    seqs_index = np.array(seqs_index)
    seqs_GC = np.array(seqs_GC)
    seqs_1hot = np.array(seqs_1hot)
    return seqs, seqs_1hot, seqs_index, seqs_GC

In [ ]:
seqs_weak, seqs_1hot_weak, seqs_index_weak, seqs_GC_weak = get_seqs(
    sites_weak, genome_file="/project/fudenber_735/genomes/mm10/mm10.fa", extent_bp=100
)
seqs_strong, seqs_1hot_strong, seqs_index_strong, seqs_GC_strong = get_seqs(
    sites_strong,
    genome_file="/project/fudenber_735/genomes/mm10/mm10.fa",
    extent_bp=100,
)

In [ ]:
def reorder_by_hamming(seqs_index, sub_index=(80, 100), verbose=False):
    """
    Reorders a set of index-encoded (A=0,C=1,G=2,T=3) DNA sequences by their similarity.
    Similarity is computedb by hamming distance for all nucleotides between
    sub_index[0]:sub_index[1].

    Returns
    --------
    reordering : np.array
        Array specifying reordering for sites.

    """

    if not (np.unique(seqs_index) == np.arange(4).astype(int)).all():
        raise ValueError("must input a 1-hot encoded DNA sequence")

    if sub_index is None:
        sub_index = (0, seqs_index.shape[1])

    seq_dist = np.zeros((seqs_index.shape[0], seqs_index.shape[0]))
    for i in range(len(seqs_index)):
        if verbose:
            if np.mod(i, 100) == 0:
                print(i)
        seq_i = seqs_index[i][sub_index[0] : sub_index[1]]
        for j in range(len(seqs_index)):
            if i < j:
                seq_j = seqs_index[j][sub_index[0] : sub_index[1]]
                seq_dist[i, j] = 1 - scipy.spatial.distance.hamming(seq_i, seq_j)
    seq_dist = seq_dist + seq_dist.T
    reordering = scipy.cluster.hierarchy.leaves_list(
        scipy.cluster.hierarchy.linkage(seq_dist)
    )
    return reordering

In [ ]:
# reorder_bp = (120,140) # when extend_bp above is 100 this corresponds to organizing by sequence after the motif
reorder_bp = (
    80,
    100,
)  # this corresponds to reordering based on sequence just before the motif
reordering = "hamming"
if reordering == "hamming":
    strong_reordering = reorder_by_hamming(seqs_index_strong, sub_index=reorder_bp)
    weak_reordering = reorder_by_hamming(seqs_index_weak, sub_index=reorder_bp)
elif reordering == "GC":
    strong_reordering = np.argsort(
        np.sum(seqs_GC_strong[:, reorder_bp[0] : reorder_bp[1]], axis=1)
    )
    weak_reordering = np.argsort(
        np.sum(seqs_GC_weak[:, reorder_bp[0] : reorder_bp[1]], axis=1)
    )
else:
    raise ValueError("must specifying a reordering")

In [ ]:
# bp_start,bp_stop = 80,100
# bp_start,bp_stop = 120,140#,100

extend_bp = 20
bp_start, bp_stop = 100 - extend_bp, 119 + extend_bp
cmap = cmaps[1]

plt_width = 4
f, axs = plt.subplots(
    figsize=(plt_width + plt_width + 2, plt_width + plt_width + 1),
    ncols=4,
    nrows=2,
    gridspec_kw={
        "height_ratios": [10, 1],
        "wspace": 0.01,
        "width_ratios": [1, 0.05, 1, 0.05],
    },
    constrained_layout=True,
)

ax = axs[0, 0]
im = ax.matshow(
    seqs_index_weak[:, bp_start:bp_stop][weak_reordering],
    cmap=cmap,
)
ax.set_title("weak DNA")
ax.set_aspect("auto")

ax = axs[0, 2]
im = ax.matshow(
    seqs_index_strong[:, bp_start:bp_stop][strong_reordering],
    cmap=cmap,
)
ax.set_title("strong DNA")
ax.set_aspect("auto")

cax = axs[0, 3]
cbar = plt.colorbar(im, cax=cax, label="DNA label", ticks=np.arange(4))
cbar.ax.set_yticklabels(["A", "C", "G", "T"])

ax = axs[1, 0]
ax.plot(np.mean(seqs_GC_weak[:, bp_start:bp_stop], axis=0))
ax.set_ylim([-0.1, 1.1])
ax.grid(True)
ax.set_xlim([0, bp_stop - bp_start])
ax.set_ylabel("avg GC content")

ax = axs[1, 2]
ax.plot(np.mean(seqs_GC_weak[:, bp_start:bp_stop], axis=0), label="weak")
ax.plot(np.mean(seqs_GC_strong[:, bp_start:bp_stop], axis=0), label="strong")
ax.set_ylim([-0.1, 1.1])
ax.grid(True)
ax.set_xlim([0, bp_stop - bp_start])
ax.legend(loc="upper center", bbox_to_anchor=(0.15, -0.35))

ax = axs[1, 1]
ax.set_visible(False)
ax = axs[0, 1]
ax.set_visible(False)
ax = axs[1, 3]
ax.set_visible(False)

plt.show()

In [ ]:
import pandas as pd

tsv_file = "/scratch1/kamulege/boundaries.motifs.ctcf.mm10.tsv"

dataframe = pd.read_csv(tsv_file, sep="\t")

print(dataframe.shape)

new_dataframe = dataframe[:100]
new_dataframe.to_csv(
    "/scratch1/kamulege/_100_boundaries.motifs.ctcf.mm10.tsv", sep="\t", index=False
)

In [ ]:
display(new_dataframe)

In [13]:
import akita_utils

df_path = "/scratch1/kamulege/test_disruption_scd_model5*/scd.h5"
# df_path =  "/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model1/scd.h5"
akita_utils.tsv_gen_utils.generate_ctcf_motifs_list(
    h5_dirs=df_path,
    rmsk_file="/project/fudenber_735/genomes/mm10/database/rmsk.txt.gz",
    jaspar_file="/project/fudenber_735/motifs/mm10/jaspar/MA0139.1.tsv.gz",
    score_key="SCD",
    mode="tail",
    num_sites=23,
)


# akita_utils.tsv_gen_utils.filter_boundary_ctcfs_from_h5(
#     h5_dirs=df_path,
#     score_key="SCD",
#     threshold_all_ctcf=5,
# )

annotating each site with boundary-wide scores
filtering sites by overlap with rmsk
filtering sites by overlap with ctcfs


['chr13,116713069,116713088,-#_genomic_SCD=32.06',
 'chr12,80724993,80725012,-#_genomic_SCD=30.0',
 'chr12,80729003,80729022,+#_genomic_SCD=29.56',
 'chr12,80722104,80722123,-#_genomic_SCD=29.55',
 'chr12,80721634,80721653,-#_genomic_SCD=29.52',
 'chr13,116712408,116712427,-#_genomic_SCD=19.28',
 'chr4,153164261,153164280,-#_genomic_SCD=10.95',
 'chr4,153166322,153166341,+#_genomic_SCD=10.68',
 'chr9,58537266,58537285,-#_genomic_SCD=9.73',
 'chr11,29717382,29717401,-#_genomic_SCD=9.5',
 'chr11,29710437,29710456,+#_genomic_SCD=9.48',
 'chr11,29714998,29715017,-#_genomic_SCD=9.41',
 'chr9,58532714,58532733,+#_genomic_SCD=9.11',
 'chr9,58534747,58534766,+#_genomic_SCD=8.92',
 'chr9,58535604,58535623,+#_genomic_SCD=7.316',
 'chr9,58534974,58534993,+#_genomic_SCD=7.254',
 'chr9,58535938,58535957,-#_genomic_SCD=7.125',
 'chr13,116713866,116713885,-#_genomic_SCD=3.98',
 'chr10,69656507,69656526,+#_genomic_SCD=3.152',
 'chr4,28817857,28817876,+#_genomic_SCD=1.305',
 'chr4,28815800,28815819,+#_

In [ ]:
job_h5_file = "/scratch1/kamulege/test_disruption_scd_model2_head1/scd.h5"
job_h5_open = h5py.File(job_h5_file, "r")

with job_h5_open as file:
    # Iterate over the keys and print the dataset names
    for key in file.keys():
        if isinstance(file.get(key), h5py.Dataset):
            print(key)

In [ ]:
import glob
import h5py

# Get a list of all HDF5 files matching the pattern
job_h5_files = glob.glob('/scratch1/kamulege/test_disruption_scd_p100_model1_head1/job*/scd.h5')

# Loop over the list of files and open each one
for job_h5_file in job_h5_files:
    job_h5_open = h5py.File(job_h5_file, "r")
    with job_h5_open as file:
        # Iterate over the keys and print the dataset names
        # print(len(job_h5_open["chrom"]))
        # for key in file.keys():
        #     if isinstance(file.get(key), h5py.Dataset):
        #         print(key)
